In [ ]:
# A rule-based sentiment analysis
#--> for more visit: https://medium.com/analytics-vidhya/rule-based-sentiment-analysis-with-python-for-turkeys-stock-market-839f85d7daaf

In [1]:
import plotly.express as px #visualization
import pandas as pd
from nltk import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
import numpy as np
import re
import nltk
import string
from evds import evdsAPI
import preprocessor as p
from snowballstemmer import TurkishStemmer
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
evds = evdsAPI('#########')
pd.set_option('display.max_colwidth', -1)


In [5]:
df=pd.read_json('/Users/umar/Desktop/bist100twits.json')
df['text1']=[p.clean(e) for e in df['text']]
df=df[['text1',  'timestamp']]
df['date']=df['timestamp'].dt.date
df['text']=df['text1'].str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13440 entries, 0 to 13439
Data columns (total 4 columns):
text1        13440 non-null object
timestamp    13440 non-null datetime64[ns]
date         13440 non-null object
text         13440 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 420.1+ KB


In [6]:
df.groupby('date')['text'].count().plot(kind="line")

In [7]:
df=df[['text1',  'timestamp']]
df['date']=df['timestamp'].dt.date
df['text']=df['text1'].str.lower()

In [8]:
neg=pd.read_csv('/Users/umar/Desktop/negative_words_tr.txt')
pos=pd.read_csv('/Users/umar/Desktop/positive_words_tr.txt')
PUNCT_TO_REMOVE = string.punctuation
stoplist=stopwords.words('turkish')
def remove_punctuation(text):# notktalama işaretleri
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df['text'] = df['text'].apply(lambda text: remove_punctuation(text))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stoplist])

df['text_stop'] = df['text'].apply(lambda text: remove_stopwords(text))

In [9]:
def stemming_tokenizer(text): 
    stemmer = TurkishStemmer()
    return [stemmer.stemWord(w) for w in word_tokenize(text)]
df['text_token'] = df['text_stop'].apply(lambda text: stemming_tokenizer(text))

In [10]:
def word_finder1(x):
    df_words = set(" ".join(x).split(" "))
    extract_words =  set(neg.eski.to_list()).intersection(df_words)
    return ', '.join(extract_words)
def word_finder2(x):
    df_words = set(" ".join(x).split(" "))
    extract_words =  set(pos.olan.to_list()).intersection(df_words)
    return ', '.join(extract_words)
df['neg'] = df.text_token.apply(word_finder1)
df['pos'] = df.text_token.apply(word_finder2)

In [11]:
df[['text_token', 'neg', 'pos']].sample(3)

,text_token,neg,pos
4613,"[y, bir, taraf, gidecek, ol, ora, …]",,
6615,"[bist30, 389, bist100, 393, fai̇z, 000]",,
3968,"[kiş, i̇talya, türki, ’, yi, virüs, sonuçlarıhat, öl, oran, aynı, ora, kıyasladık, görüyoruzkriz, beslene, kes, bellik, nakitteky, düşünenyol, göstere, parmak, say, kadarbu, mubarek, gece, kyi̇ç, istiyor, allah, kat, on, nasip, et]",virüs,


In [12]:
df['len_pos'] = df.pos.apply(lambda x: len(x.split(', ')))
df['len_neg'] = df.neg.apply(lambda x: len(x.split(', ')))

In [13]:
df['len_neg']=df['len_neg']-1
df['len_pos']=df['len_pos']-1

In [14]:
df['text_count'] = df['text_stop'].str.split().apply(len).value_counts()
df['polarity'] = df['len_pos'] - df['len_neg']
df['sent_rate']=(df['len_pos'] + df['len_neg'])/ df['text_count']

In [17]:
d=pd.DataFrame(df.groupby('date')['polarity'].mean())
d=d.reset_index()
d['date']=pd.to_datetime(d['date'])
d=d.sort_values(by='date')
fig = px.line(d, x=d.date, y=d.polarity.rolling(window=4).mean(), title='average daily polarity')
fig.show()

In [19]:
bist=evds.get_data(['TP.MK.F.BILESIK'] , startdate="01-03-2020", enddate='01-05-2020')
bist.columns=['tarih', 'kapanış']
fig = px.line(bist, x=bist.tarih, y=bist.kapanış, title='BIST100 Index in the same date range')
fig.show()